In [1]:
import pandas as pd
import numpy as np
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import datetime as dt

import re, string, unicodedata
import nltk
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

from api_keys import api_key

In [2]:
def request_song_info(artist_id, page_number):
    base_url = f'https://api.genius.com'
    headers = {f'Authorization': f'Bearer {api_key}'}
    search_url = f'{base_url}/artists/{artist_id}/songs?per_page=40&page={page_number}&sort=popularity'
    response = requests.get(search_url, headers=headers)

    return response

In [3]:
jay_z_dict = {}
beyonce_dict = {}
artist_dict = {2 : jay_z_dict, 498 : beyonce_dict}
        
for artist_id, artist_dict in artist_dict.items():      

    for page in range (1, 20):
        response = request_song_info(artist_id, page)
        json = response.json()
        remote_song_info = None

        for song in json['response']['songs']:
            if str(artist_id) in str(song['primary_artist']['id']):
                remote_song_info = song

                if remote_song_info:
                    song_url = remote_song_info['url']

                if len(artist_dict) == 300:
                    break

                else:
                    artist_dict.update({song['title'].lower() : song_url})

In [4]:
print(f'the jay_z_dict contains {len(jay_z_dict)} songs.')
print(f'the beyonce_dict contains {len(beyonce_dict)} songs.')

the jay_z_dict contains 300 songs.
the beyonce_dict contains 300 songs.


In [5]:
def scrap_song_url(song_url, date_index):
    page = requests.get(song_url)
    html = BeautifulSoup(page.text, 'html.parser')
    release_date = dt.datetime.strptime(html.find_all('span', {'class': 'metadata_unit-info metadata_unit-info--text_only'})[date_index].get_text(), '%B %d, %Y').strftime('%Y%m%d')
    lyrics = html.find('div', class_='lyrics').get_text()

    return release_date, lyrics

In [6]:
relationship_start = scrap_song_url(jay_z_dict["'03 bonnie & clyde"], 1)[0]
print(int(relationship_start))

20021010


In [7]:
pre_jay_z_dict = {}
post_jay_z_dict = {}

for song, song_url in jay_z_dict.items():
    try:
        if int(scrap_song_url(song_url, 1)[0]) >= int(relationship_start):
            if len(post_jay_z_dict) == 50:
                continue
            else:
                post_jay_z_dict.update({song : scrap_song_url(song_url, 1)[1]})
        else:
            if len(pre_jay_z_dict) == 50:
                continue
            else:
                pre_jay_z_dict.update({song : scrap_song_url(song_url, 1)[1]})

    except:
        try:
            if int(scrap_song_url(song_url, 0)[0]) >= int(relationship_start):
                if len(post_jay_z_dict) == 50:
                    continue
                else:
                    post_jay_z_dict.update({song : scrap_song_url(song_url, 0)[1]})
            else:
                if len(pre_jay_z_dict) == 50:
                    continue
                else:
                    pre_jay_z_dict.update({song : scrap_song_url(song_url, 0)[1]})                
                
        except:
            print(f'FAILED on {song}')

FAILED on dead presidents iii
FAILED on history
FAILED on 44 four's
FAILED on a million and one questions (remix)
FAILED on blow the whistle
FAILED on 30 something (remix)
FAILED on open letter part ii (remix)
FAILED on pump it up freestyle
FAILED on if i can't (freestyle)
FAILED on when the money goes
FAILED on addicted to the game
FAILED on 2002 funk flex freestyle
FAILED on dead or alive, pt. 1
FAILED on early this morning


In [8]:
print(len(pre_jay_z_dict))
print(len(post_jay_z_dict))

50
50


In [52]:
pre_jay_z_df = pd.DataFrame.from_dict(pre_jay_z_dict, orient = 'index', columns = ['pre_text'])
post_jay_z_df = pd.DataFrame.from_dict(post_jay_z_dict, orient = 'index', columns = ['post_text'])

In [53]:
pre_jay_z_df.head()

,pre_text
still d.r.e.,"\n\n[Produced by Dr. Dre, Scott Storch and Mel..."
renegade,\n\n[Verse 1: JAY-Z]\nMotherfuckers say that I...
takeover,\n\n[Produced by Kanye West]\n\n[Hook]\nR.O.C....
song cry,\n\n[Produced by Just Blaze]\n\n[Intro: Bobby ...
big pimpin',\n\n[Produced by Timbaland]\n\n[Intro: Jay-Z]\...


In [54]:
pre_jay_z_df.pre_text = pre_jay_z_df.pre_text.apply(lambda x: x.replace('\n', ' '))

In [55]:
pre_jay_z_df.pre_text = pre_jay_z_df.pre_text.apply(lambda x: re.sub('\[[^]]*\]', '', x))

In [56]:
pre_jay_z_df.pre_text = pre_jay_z_df.pre_text.apply(lambda x: x.lower())

In [57]:
pre_jay_z_df.pre_text = pre_jay_z_df.pre_text.apply(lambda x: contractions.fix(x))

In [58]:
pre_jay_z_df.pre_text = pre_jay_z_df.pre_text.apply(lambda x: re.sub(r'[^\w\s]','', x))

In [61]:
# pre_jay_z_df.pre_text = pre_jay_z_df.pre_text.apply(lambda x: nltk.word_tokenize(x))

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  Searched in:
    - 'C:\\Users\\jgross/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\jgross\\Anaconda3\\nltk_data'
    - 'C:\\Users\\jgross\\Anaconda3\\share\\nltk_data'
    - 'C:\\Users\\jgross\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\jgross\\AppData\\Roaming\\nltk_data'
    - ''
**********************************************************************


In [60]:
pre_jay_z_df.to_csv('test.csv')